In [25]:
import pandas as pd
import requests

In [26]:
visits_df = pd.read_csv('visits_1k.csv')
regs_df = pd.read_csv('regs_1k.csv')

In [27]:
visits_df.head()

,uuid,platform,user_agent,date
0,1de9ea66-70d3-4a1f-8735-df5ef7697fb9,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,2023-03-01T13:29:22
1,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) Apple...,2023-03-01T16:44:28
2,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) Apple...,2023-03-06T06:12:36
3,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109...,2023-03-01T20:16:37
4,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109...,2023-03-05T17:42:47


In [28]:
visits_df.describe()

,uuid,platform,user_agent,date
count,1000,1000,1000,1000
unique,519,3,28,996
top,251a0926-ece3-4d77-aa42-ab569fdf9fe2,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01T08:01:45
freq,4,954,71,2


In [29]:
regs_df.head()

,date,user_id,email,platform,registration_type
0,2023-03-01T00:25:39,8838849,joseph95@example.org,web,google
1,2023-03-01T14:53:01,8741065,janetsuarez@example.net,web,yandex
2,2023-03-01T14:27:36,1866654,robert67@example.org,web,google
3,2023-03-01T02:42:34,1577584,elam@example.net,web,apple
4,2023-03-01T10:27:14,4765395,stephanie68@example.net,web,yandex


In [30]:
regs_df.describe()

,user_id
count,1.000000e+03
mean,4.488623e+06
std,2.620568e+06
min,2.236800e+04
25%,2.235489e+06
50%,4.473044e+06
75%,6.779707e+06
max,8.881772e+06


In [31]:
rsp_visits = requests.get('https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01')
visits_data = rsp_visits.json()
visits_df = pd.DataFrame(visits_data)

In [32]:
rsp_regs = requests.get('https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01')
data_regs = rsp_regs.json()
regs_df = pd.DataFrame(data_regs)

In [42]:
regs_df['datetime'] = pd.to_datetime(regs_df['datetime'], format='%Y-%m-%dT%H:%M:%S')
visits_df['datetime'] = pd.to_datetime(visits_df['datetime'], format='%Y-%m-%dT%H:%M:%S')

In [49]:
regs_df['date'] = regs_df['datetime'].dt.date
visits_df['date'] = visits_df['datetime'].dt.date

In [54]:
regs_grouped = regs_df.groupby(['date', 'platform']).size().reset_index(name='registrations')
visits_grouped = visits_df.groupby(['date', 'platform']).size().reset_index(name='visits')

In [60]:
merged_df = pd.merge(regs_grouped, visits_grouped, on=['date', 'platform'], how='outer').fillna(0)
merged_df['date'] = pd.to_datetime(merged_df['date'])

In [65]:
merged_df['conversion'] = merged_df['registrations'] / merged_df['visits'].replace({0: 1}) * 100

In [66]:
merged_df['date_group'] = (merged_df['date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')

In [67]:
merged_df.drop(columns='date', inplace=True)

In [69]:
merged_df.rename(columns={'platform': 'platform'}, inplace=True)

In [71]:
merged_df.to_json('./conversion.json', orient='columns')